In [58]:
import ddddocr
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pickle
from concurrent.futures import ThreadPoolExecutor
import cv2
import numpy as np
import io
from PIL import Image
import keyboard
from IPython.display import clear_output
import re

In [37]:
def refresh_cookies():
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    driver.maximize_window()
    driver.get(f'https://tixcraft.com')
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='close-alert']"))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-reject-all-handler']"))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li[class='account-login text-center dropdown register-link d-nonex'] > a"))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[id='loginGoogle']"))).click()
    while(True):
        key = input("After login and finished all tasks, wait a few seconds and enter 'y' to save cookies\n")
        if(key == 'y'): break
    pickle.dump(driver.get_cookies(), open(f"tixcraft_cookies.pkl", "wb"))
    print('Cookies saved')
    driver.quit()
# refresh_cookies()

In [38]:
class StartUPTasks:
    def __init__(self) -> None:
        global driver
        print('Selected activity:', driver.find_element(By.CSS_SELECTOR, "h1").text)

    def adjust_window(self):
        driver.set_window_position(2000, 0)
        driver.maximize_window()
    
    def add_cookies(self):
        driver.delete_all_cookies()
        cookies = pickle.load(open("tixcraft_cookies.pkl", "rb"))
        for cookie in cookies:
            driver.add_cookie(cookie)

    def close_alert_button(self):
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='close-alert']"))).click()
    
    
    def execute_tasks(self):
        with ThreadPoolExecutor() as executor:
            executor.submit(self.close_alert_button)
            executor.submit(self.add_cookies)
            executor.submit(self.adjust_window)
        driver.refresh()


In [39]:
# found_matched = False
# while(not found_matched):
#     clear_output(wait=True)
#     if(keyboard.is_pressed('y')):
#         break
#     for event_datetime, event_name, event_venue in event_priority:
#         event_table = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "tbody > tr")))
#         found_matched = False
#         find_tickets_success = False
#         for event_row in event_table:
#             event_cell = WebDriverWait(event_row, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "td")))
#             datetime_match, name_match, venue_match = True, True, True
#             if(event_datetime): 
#                 datetime_match = (event_cell[0].text == event_datetime)
#             if(event_name): 
#                 name_match = (event_cell[1].text == event_name)
#             if(event_venue): 
#                 venue_match = (event_cell[2].text == event_venue)
#             if(datetime_match and name_match and venue_match):
#                 found_matched = True
#                 matched_event_status = WebDriverWait(event_cell[-1], 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "*")))
#                 if(len(matched_event_status) == 2):
#                     print(f'{event_datetime} status: {matched_event_status[1].text}')
#                 elif(matched_event_status[0].tag_name == "div"):
#                     print(f'{event_datetime} status: {matched_event_status[0].text}')
#                 else:
#                     find_tickets_success = True
#                     print(f"Selected event: {event_datetime} {event_name} {event_venue}")
#                     while(True):
#                         try:
#                             matched_event_status[0].click()
#                             break
#                         except:
#                             pass
#             if(found_matched): 
#                 break
#         if(find_tickets_success): 
#             break
#     if(not found_matched):
#         print("No available event found, refreshing...")
#         driver.refresh()

In [40]:
def event_selection(time_name_venue_status):
    global event_priority
    datetime_e, name_e, venue_e, status_e = [cell.text for cell in time_name_venue_status]
    datetime_match, name_match, venue_match = True, True, True
    for i, (datetime_p, name_p, venue_p) in enumerate(event_priority):
        if(datetime_p):
            datetime_match = (datetime_e == datetime_p)
        if(name_p):
            name_match = (name_e == name_p)
        if(venue_p):
            venue_match = (venue_e == venue_p)
        if(datetime_match and name_match and venue_match):
            if(status_e == "Find tickets"):
                return i
            else:
                print(f"{datetime_e}, {name_e}, at {venue_e} not available")
    return 128


In [41]:
def event_selection_page():
    global driver
    found_matched_event = False
    while(not found_matched_event):

        if(keyboard.is_pressed('y')):
            break

        event_table = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "tbody > tr")))
        event_rows = [WebDriverWait(row, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "td"))) for row in event_table]
        event_selection_check = np.ones(len(event_rows)) * 128

        with ThreadPoolExecutor() as executor:
            event_selection_result = [priority_idx for priority_idx in executor.map(event_selection, event_rows)]

        if(not np.all(event_selection_result == event_selection_check)):
            event_idx = np.argmin(event_selection_result)
            datetime, name, venue, status = event_rows[event_idx]
            print(f"Found event: {datetime.text}, {name.text}, at {venue.text}")
            found_matched_event = True
            while(True):
                try:
                    status.click()
                    break
                except:
                    pass
        else:
            print("No available tickets, refreshing...")
            driver.refresh()


In [42]:
# section_li_lst = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='zone area-list'] > ul > li")))
# section_name_status = [s.text.strip().split(' ', 1) for s in section_li_lst]
# found_matched_section = False
# n = 0
# while(not found_matched_section and n < 5):
#     for priority_keyword in wanted_section_keywords_priority:
#         for i, (name, status) in enumerate(section_name_status):
#             exact_match = True
#             for keyword in priority_keyword:
#                 if(keyword not in name):
#                     exact_match = False
#                     break
#             if(exact_match and status != "Sold out"):
#                 if("seat(s) remaining" in status and int(status.split(' ')[0]) < ticket_quantity):
#                     break
#                 else:
#                     print(f"Found tickets: {name}, {status}")
#                     while(True):
#                         try:
#                             # section_li_lst[i].click()
#                             break
#                         except:
#                             pass
#                     found_matched_section = True
#                     break
#         if(found_matched_section): 
#             break
#     if(not found_matched_section):
#         n += 1
#         driver.refresh()


In [43]:
def section_selection(name_status):
    global wanted_section_keywords_priority, ticket_quantity
    name, status = name_status
    for i, section_keywords in enumerate(wanted_section_keywords_priority):
        exact_match = True
        for keyword in section_keywords:
            if(keyword not in name):
                exact_match = False
                break
        if(exact_match and ("Sold out" not in status)):
            if("seat(s) remaining" in status):
                if(int(re.search(r"([0-9]) seat\(s\) remaining", status).group(1)) < ticket_quantity):
                    break
            return i
    return 128

In [64]:
def seat_selection_page():
    global driver   
    found_matched_section = False
    while(not found_matched_section):

        if(keyboard.is_pressed('y')):
            break
        clear_output(wait=True)
        
        section_li_lst = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='zone area-list'] > ul > li")))
        section_name_status = [s.text.strip().split(' ', 1) for s in section_li_lst]
        section_selection_check = np.ones(len(section_name_status)) * 128

        with ThreadPoolExecutor() as executor:
            section_selection_result = [priority_idx for priority_idx in executor.map(section_selection, section_name_status)]
        driver.find_element(By.TAG_NAME, 'html').send_keys(Keys.END)
        if(not np.all(section_selection_result == section_selection_check)):
            section_idx = np.argmin(section_selection_result)
            print(f"Found tickets: {section_name_status[section_idx][0]}, {section_name_status[section_idx][1]}")
            found_matched_section = True
            while(True):
                try:
                    section_li_lst[section_idx].click()
                    break
                except:
                    pass
        else:
            driver.refresh()

In [55]:
def enter_captcha():
    def guess_captcha():
        global driver
        ocr = ddddocr.DdddOcr(show_ad=False)
        while(True):
            captcha = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "img[id='TicketForm_verifyCode-image']")))
            img = np.array(Image.open(io.BytesIO(captcha.screenshot_as_png)).convert('L'))
            # with open('captcha.png', 'wb') as f:
            #     f.write(captcha.screenshot_as_png)
            if(img.shape == (100, 120)):
                break
        for kernel_size in [3, 4, 2]:
            img_erode = cv2.erode(img, np.ones((kernel_size, kernel_size), np.uint8))
            # cv2.imwrite('captcha_processed.png', img_erode)
            # with open('captcha_processed.png', 'rb') as f:
            #     captcha_img = f.read()
            guess = ocr.classification(Image.fromarray(img_erode))
            if(len(guess) == 4):
                break
        if(len(guess) != 4):
            guess = ocr.classification(Image.fromarray(img))
        return guess
    try:
        captcha_guess = guess_captcha()
        verify_code_box = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='TicketForm_verifyCode']")))
        verify_code_box.clear()
        verify_code_box.send_keys(captcha_guess)
    except Exception as e:
        print(e)
        print(f'Enter captcha failed with {e}')

def agree_tos():
    global driver
    agree_tos_checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='TicketForm_agree']")))
    while(not agree_tos_checkbox.is_selected()):
        try: 
            agree_tos_checkbox.click() 
        except: 
            pass

def select_quantity():
    global driver, ticket_quantity
    full_ticket = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select")))
    possible_quantity = WebDriverWait(full_ticket, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "option")))
    final_quantity = min(ticket_quantity, len(possible_quantity) - 1)
    choose_quantity = possible_quantity[final_quantity]
    try:
        while(not choose_quantity.is_selected()):
            choose_quantity.click()
    except Exception as e:
        print(f'Select quantity failed with {e}')

def get_submit_button():
    global driver
    return WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='btn btn-primary btn-green']")))

In [46]:
def ticket_qty_page():
    with ThreadPoolExecutor() as executor:
        executor.submit(agree_tos)
        executor.submit(select_quantity)
        executor.submit(enter_captcha)
        submit_button = executor.submit(get_submit_button).result()
    while(True):
        try:
            submit_button.click()
            break
        except:
            pass


In [65]:
def decide_current_page_action():
    global driver, url
    if(driver.current_url == url):
        event_selection_page()
        return 1
    else:
        page_name = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ol > li")))[-1].text    
        if(page_name == "Seat Selection"):
            seat_selection_page()
            return 1
        elif(page_name == "Ticket Qty."):
            ticket_qty_page()
            return 10
        else:
            return 0
    

In [61]:
ticket_quantity = 2
event = '25_maroon5'
url = f'https://tixcraft.com/activity/game/{event}'
event_priority = [
    [None, "Maroon 5 Asia 2025 – Kaohsiung", None],
]
wanted_section_keywords_priority = [
    ['A13'], 
    ['A14'], 
    ['A15'], 
    ['A16'],
    ['A7'],
    ['A8'],
    ['A1區'],
    ['A2區'],
    ['B1區'],
    ['B5區'],
]
print(f"Wanting to purchase \"{ticket_quantity}\" tickets")

Wanting to purchase "2" tickets


In [49]:
options = uc.ChromeOptions()
driver = uc.Chrome(options=options)
driver.get(url)
tasks = StartUPTasks()
tasks.execute_tasks()

Selected activity: Maroon 5 Asia 2025 – Kaohsiung


In [66]:
while(True):
    if(keyboard.is_pressed('y')):
        break
    wait_seconds = decide_current_page_action()

    try:
        WebDriverWait(driver, wait_seconds).until(EC.alert_is_present()).accept()
    except:
        pass


Found tickets: B5區, 4800 8 seat(s) remaining
